Imports

In [2]:
import cv2
import numpy as np
import pickle
import cv2
import math

Loading Models


In [5]:
import cv2
import joblib

In [6]:
age_model = joblib.load(open('age_model.pkl', 'rb'))
meNotme_model = joblib.load(open('model_binary.pkl', 'rb'))
multi_model = joblib.load(open('multi_label_model.pkl', 'rb'))


ValueError: Layer 'conv2d_6' expected 2 variables, but received 0 variables during loading. Expected: ['conv2d_6/kernel:0', 'conv2d_6/bias:0']

Face Detection in Frame

In [4]:
# Binds face in the frame with a box
# If face is detected it returns an image with the face bound, and the pixel area inside the box, excluding the actual boundary
# If there are multiple faces in frame, it chooses the one having the greatest area in frame

def faceDetect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(64, 64)
    )
    
    if(len(faces) > 1):
        cur_max = 0
        face = 0

        for (x, y, w, h) in faces:
            length = math.sqrt((x - (x+w))**2 + (y - (y+h))**2 )
            
            if length > cur_max:
                cur_max = length
                face = [x,y,w,h]

        faces = np.array([face])

    for (x, y, w, h) in faces:
    
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2) # Draw a rectangle around the detected area (face)

        roi_color = image[y+2:y+2 + h-3, x+2:x+2 + w-3] # The pixel area containing the face

        return True,[image,roi_color]
    
    return False,[]

Camera Predictions

In [5]:
# Starts the camera, provides each frame to the faceDetect() function
# If a face is detected it prints the Identity(Sameed or Unknown) and Age using Linear and SGD regression at the bottom of the screen
# If no face detected, it prints appropriately
# Use esc to exit

cv2.destroyAllWindows()
cam = cv2.VideoCapture(0)

while True:

    check, frame = cam.read()
    frame = cv2.flip(frame,1)

    
    frame = cv2.copyMakeBorder(frame,1,350,1,200,cv2.BORDER_CONSTANT,value=0)

    cv2.putText(frame,"Who:",(20,530),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
    cv2.putText(frame,"Age:",(20,630),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)

    cv2.imshow('video', frame)
    detected,bound_frame = faceDetect(frame)


    if(detected):
        frame = bound_frame[0]
        cropped_frame = bound_frame[1]

        img_resize_clf = cv2.resize(cropped_frame, (64,64)) # The classifier was trained on 64x64 images
        img_resize_reg = cv2.resize(cropped_frame, (32,32)) # The regressors were trained on 32x32 images

        cur_image_clf = img_resize_clf.flatten()
        cur_image_ref = img_resize_reg.flatten()

        me_pred = MeClf_model.predict([cur_image_clf])
        y_pred_proba = MeClf_model.predict_proba([cur_image_clf])

        ols_pred = ridge_model.predict([cur_image_ref])        

        sgd_pred = SGD_model.predict([cur_image_ref])

        cv2.putText(frame,"Ridge - " + str(int(ols_pred)),(120,630),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1)
        cv2.putText(frame,"SGD - " + str(int(sgd_pred)),(120,670),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1)

        if(me_pred == 0):
            # if(y_pred_proba[:5] != None):
            #     text = "Unknown - "
            # else:
            text = "Unknown - " + str(y_pred_proba[0])
            print(text)

            cv2.putText(frame,text,(20,580),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
            # cv2.imshow('video', frame)

        if(me_pred == 2):
            # if(y_pred_proba[:5] != None):
            #     text = "Unknown - "
            # else:
            text = "Arham - " + str(y_pred_proba[0])
            print(text)

            cv2.putText(frame,text,(20,580),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
            # cv2.imshow('video', frame)

        else:
            # if(y_pred_proba[:5] != None):
            #     text = "Sameed - "
            # else:
            text = "Sameed - " + str(y_pred_proba[0])
            print(text)
            cv2.putText(frame,text,(20,580),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
            
            # cv2.imshow('video', frame)

    else:
        cv2.putText(frame,"Face Not Detected",(20,580),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
        # cv2.imshow('video', frame)

    cv2.imshow('video', frame)


    key = cv2.waitKey(1)
    if key == 27:
        break

cam.release()
cv2.destroyAllWindows()

Sameed - [1.09463713e-12 9.99999959e-01 4.13901933e-08]
Sameed - [7.37106572e-13 9.99999981e-01 1.93296715e-08]
Sameed - [2.27077399e-13 9.99999987e-01 1.30911479e-08]
Sameed - [1.19737133e-13 9.99999990e-01 1.03132255e-08]
Sameed - [2.50340649e-14 9.99999998e-01 2.30809063e-09]
Sameed - [2.66230354e-14 9.99999998e-01 1.96998386e-09]
Sameed - [4.57358083e-14 9.99999997e-01 3.20846708e-09]
Sameed - [1.38036734e-14 9.99999999e-01 5.78304198e-10]
Sameed - [1.29862560e-14 9.99999999e-01 1.08757033e-09]
Sameed - [1.80394658e-14 9.99999995e-01 4.52622332e-09]
Sameed - [6.80968650e-15 1.00000000e+00 4.11795765e-10]
Sameed - [1.41928322e-14 9.99999997e-01 3.38537633e-09]
Sameed - [8.80325389e-15 9.99999999e-01 6.80346845e-10]
Sameed - [8.52990397e-15 9.99999999e-01 6.06991576e-10]
Sameed - [2.06707280e-13 9.99999996e-01 3.50231074e-09]
Sameed - [8.10694544e-13 9.99999998e-01 1.92381432e-09]
Sameed - [1.09438544e-12 9.99999990e-01 9.56100262e-09]
Sameed - [5.46174934e-12 9.99999979e-01 2.063856

In [6]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")

demo.launch()

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
